# Interactive Notebook: Network Assessment for Active Mobility

This Jupyter-Notebook guides through several network assessments based on NetAScore output.
In order to execute this Notebook, please make sure to have all requirements fulfilled - please follow the instructions in [README.md](README.md).

If you do not have a NetAScore output file at hand, please download one of the example files from https://doi.org/10.5281/zenodo.10886962 and place it inside the subdirectory `NetAScore/data/`.

In [ ]:
# import required packages
import geopandas as gpd
import networkx as nx
import os

# settings
case_id = "at_salzburg"

# computed properties
net_file = os.path.join("NetAScore", "data", f"netascore_{case_id}.gpkg")

## Load the network and start exploring bikeability...
Here, we load the network which was processed by NetAScore and output the first 10 rows (edges).

In [ ]:
net = gpd.read_file(net_file, layer="edge")
net.head()

Let's filter the network to only show segments with high bikeability and display the result on a map.

In [ ]:
net[net.index_bike_ft > 0.75].explore()

We can also choose a different basemap with reduced color scheme:
(see online documentation for more options: [GeoDataFrame.explore()](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html))

In [ ]:
net[net.index_bike_ft > 0.75].explore(tiles="CartoDB Positron")

...and we can further apply a color scheme to plot the high-bikeability according to their bikeability value:

In [ ]:
net[net.index_bike_ft > 0.75].explore(column="index_bike_ft", cmap="Blues", tiles="CartoDB Positron")

## Let's prepare our network for routing...
First, we need to generate a directed graph from our input data. Having an input edge (A<->B) with an index_ft and index_tf will result in two rows: (A->B, index_ft) and (B->A, index_tf). 
Furthermore, we want to not only compute shortest paths, but allow for computing bikeable routes as well. For this, we need to compute a routing cost per segment. This can be regarded as perceived distance: better suitability means perceived distance is close to real distance, low suitability results in perceived distance is (1+ROUTING_FACTOR) * length (with default setting: up to five times real segment length).
All of these steps are outsourced into a function `netascore_to_routable_net` which is defined in [algo/net_helper.py](algo/net_helper.py).

To execute this step, we simply call the function as follows:

In [ ]:
import algo.net_helper as nh
net_routing = nh.netascore_to_routable_net(net)

Next, we have to create a NetworkX graph object from our edge list representation of the network.
We use a `MultiDiGraph` to allow for multiple directed links between nodes:

In [ ]:
g = nx.from_pandas_edgelist(net_routing, source='from_node', target='to_node', 
                            edge_attr=True, create_using=nx.MultiDiGraph, edge_key="edge_id")
g

## Load and display the nodes layer
As basis for computing routes between pairs of nodes of the network, we will first load and display the nodes layer.

In [ ]:
nodes = gpd.read_file(net_file, engine='pyogrio', fid_as_index=True, layer="node")
#nodes = gpd.read_file(net_file, layer="node")
#nodes["id"] = nodes.index + 1
nodes.explore(tiles="CartoDB Positron", marker_kwds={"radius":2}, style_kwds={"stroke":False})

Next, we can manually select an origin node as well as a destination node from the interactive map and assign the respective nodeIDs to the variables `from_node` and `to_node`.

In [ ]:
from_node = 21181  #20741
to_node = 23302  #25235

## Compute routes
Now, we can compute routes...
### General routing approach

In [ ]:
npath = nx.shortest_path(g, from_node, to_node) # this call results in computing the shortest (lowest number of passed segments) path
npath

For visualization, we need to retrieve an edge sequence from the node sequence returned by `nx.shortest_path`:

In [ ]:
# get edge sequence (edge IDs)
def get_epath(nseq):
    ep = []
    for i in range(len(nseq)-1):
        dta = g.get_edge_data(nseq[i], nseq[i+1])
        eid = list(dta.keys())[0]
        ep.append(eid)
    return ep

epath = get_epath(npath)
epath

In [ ]:
m = net.loc[epath].explore(tiles="CartoDB Positron")
nodes.loc[[from_node, to_node]].explore(m=m, color=["green", "red"], marker_kwds={"radius":5}, style_kwds={"weight":2})

### Shortest distance path
In order to retrieve the shortest distance path, we need to specify the edge weight column - in this case we use the "length" attribute.

In [ ]:
shortest_dist_path = get_epath(nx.shortest_path(g, from_node, to_node, weight="length"))
m = net_routing.loc[shortest_dist_path].explore(tiles="CartoDB Positron", tooltip=["cost_bike_ft"])
nodes.loc[[from_node, to_node]].explore(m=m, color=["green", "red"], marker_kwds={"radius":5}, style_kwds={"weight":2})

### Bikeable path
Here, we use the `cost_bike_ft` column which contains our pre-computed "perceived distance" value.

In [ ]:
bikeable_path = get_epath(nx.shortest_path(g, from_node, to_node, weight="cost_bike_ft"))
m = net.loc[bikeable_path].explore(tiles="CartoDB Positron")
nodes.loc[[from_node, to_node]].explore(m=m, color=["green", "red"], marker_kwds={"radius":5}, style_kwds={"weight":2})

## Additionally retrieve edge length and bikeability values
To allow retrieving additional attributes, we need to keep track of the direction of edge traversal in addition to the edge ID.

In [ ]:
import pandas as pd
# get edge sequence including "inverted" attribute (direction of traversal)
def get_epath_dir(nseq):
    ep = []
    inv = []
    for i in range(len(nseq)-1):
        dta = g.get_edge_data(nseq[i], nseq[i+1])
        eid = list(dta.keys())[0]
        ep.append(eid)
        inv.append(dta[eid]["inv"])
        #print(eid,dta[eid]["inv"])
    return pd.DataFrame(data={"edge_id":ep, "inv":inv})


In [ ]:
bp = get_epath_dir(nx.shortest_path(g, from_node, to_node, weight="cost_bike_ft"))
bp.head(5)

Now we have to prepare the "net_routing" DataFrame for allowing join operations with the routing result. For this, we now need to have a MultiIndex, allowing to jointly query/filter based on `edge_id` and `inv`.

In [ ]:
net_routing.head(5)

In [ ]:
net_join = net_routing.set_index(["edge_id", "inv"])
net_join.head(5)

In the following step we can join the path result with the just prepared network data by the joint key, using the edge ID and direction (inv).

In [ ]:
bp_joined = net_join.join(bp.set_index(["edge_id", "inv"]), how="right")
bp_joined.head(5)

This now allows us to plot additional route characteristics - here, the bikeability of each segment:

In [ ]:
m = bp_joined.explore(tiles="CartoDB Positron", column="index_bike_ft", cmap="RdYlGn", vmin=0, vmax=1)
nodes.loc[[from_node, to_node]].explore(m=m, color=["green", "red"], marker_kwds={"radius":5}, style_kwds={"weight":2})

...and we can compute additional path statistics:

In [ ]:
def get_path_stats(path_df):
    length = path_df["length"].sum()
    length_weighted_avg_index = (path_df.index_bike_ft * path_df.length).sum() / path_df.length.sum()
    min_index = path_df.index_bike_ft.min()
    max_index = path_df.index_bike_ft.max()
    return {"total_length": length, "length_weighted_avg_index": length_weighted_avg_index, 
            "min_index": min_index, "max_index": max_index}

In [ ]:
get_path_stats(bp_joined)

## Make the routing part re-usable

In [ ]:
def routing(from_node:int, to_node:int, compute_bikeable_path:bool, display_route:bool):
    weight_col = "cost_bike_ft"
    if not compute_bikeable_path:
        weight_col = "length"
    path = get_epath_dir(nx.shortest_path(g, from_node, to_node, weight=weight_col))
    net_join = net_routing.set_index(["edge_id", "inv"])
    path_joined = net_join.join(path.set_index(["edge_id", "inv"]), how="right")
    if display_route:
        m = path_joined.explore(tiles="CartoDB Positron", column="index_bike_ft", cmap="RdYlGn", vmin=0, vmax=1)
        display(nodes.loc[[from_node, to_node]].explore(m=m, color=["green", "red"], marker_kwds={"radius":5}, style_kwds={"weight":2}))
    return get_path_stats(path_joined)

In [ ]:
# compute shortest path
routing(from_node, to_node, False, True)

In [ ]:
#compute bikeable path
routing(from_node, to_node, True, True)